#The problem

**Question:** Find how many numbers exist such that the number is at most 16 digits, and has *at least* 4 distinct prime factors, where the primes are less than 100.

**Disclaimer:** Since uploading this, I have realized that my solution is incomplete. It is headed in the correct direction, but I don't fully apply the inclusion-exclusion principle as I should. I'll fix that when I get the time.

So let's think about that. Here are some things to keep in mind:

- We have to find **how many** numbers like that there are, not the numbers themselves. Computing the numbers themselves will be too computationally expensive.
- For a number to be at most 16 digits, it has to be less than `10**16`.
- If we have a number which has 4 prime factors, then multiples of that number will also have the same factors. (This is simply a property of factoring. If 2 is a factor of 4, it is also a factor of all the multiples of 4, namely: 4, 8, 12, 16...)

Let's write some setup code.

In [1]:
import math
import itertools

count = 0
limit = 10**16 - 1 # No qualifying number should be above this limit
prime_limit = 100 # No prime factor should be above this limit
primes = [] # Used to store all the primes smaller than the limit

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
       if n % i == 0:
         return False
    return True


def get_primes(p_limit):
    for i in range(2, p_limit):
       if is_prime(i):
         primes.append(i) # add it to the global list we have

def lcm(a, b):
    small = min(a, b)
    # We start at the smaller number, and take steps of that size since the LCM has to be a multiple of this number
    # Given that, we know that the biggest LCM we can find is simply a * b, which is why that is the
    # end of the range
    for i in range(small, a * b + 1, small):
       if i % b == 0 and i % a == 0:
         return i


Given that code, which should be fairly simple to understand, let's just test it and make sure we get values that we expect out of it

In [2]:
assert is_prime(5)
assert is_prime(101)
assert is_prime(1200) == False
assert is_prime(1) == False

assert lcm(3, 6) == 6
assert lcm(3, 4) == 12
assert lcm(5, 7) == 35
assert lcm(6, 9) == 18

Alright, our basic helper functions work. Let's try to figure the problem out now. The key thing to understand is that if a `N` number has `a, b, c, d` as prime factors, then `2 * N` will also have the same primes as factors. So will `3 * N`. We can follow the pattern to get as many numbers as we want which have `a, b, c, d` as prime factors.

However, we have a condition to satisfy. These numbers cannot be more than 16 digits. So we need to know how many multiples of `N` there are, that are less than `limit`. If you think about this hard enough, you'll see that the answer is simply `limit // N` where `//` represents integer division.

For example, how many multiples of 4 are there below 10?

In [3]:
10 // 4 # Gives 2, because the list is: 4, 8

2

There is a problem, here. The following code should illustrate that:

In [4]:
12 // 4

3

That gives us 3, which means we're finding how many multiples of 4 there are that are less than *or equal* to 12. Fixing it is simple. We just run `11 // 4` instead. You'll note that this is why we used `limit = 10**16 - 1` in our starter code.

Anyway, let's get back to the problem. Given `N`, we can find how many multiples of N there are under the limit. Using this fact, we can simply find all the possible `N`'s we can form with our primes, and then we can just add `limit // current_N` to the count. Let's get around to doing that.

First, we'll get our list of primes.

In [5]:
get_primes(prime_limit)
print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


Let's see how we can use `itertools.combinations` to create unique combinations of sequences. The following line of code *chooses* 2 elements out of the list `[1, 2, 3]` to create the appropriate unique combinations.

In [6]:
print(list(itertools.combinations([1,2,3], 2)))

[(1, 2), (1, 3), (2, 3)]


Similarly, we can create a list of all the possible ways there are to combine 4 primes. For example:

In [7]:
print(list(itertools.combinations([3, 5, 7, 11], 2)))

[(3, 5), (3, 7), (3, 11), (5, 7), (5, 11), (7, 11)]


If we multiply the elements of each tuple in the list above, we'll get a list of numbers which have 2 prime factors. This will be the list of `N`s that we required. Let's write code to do that.

In [8]:
prime_combinations = itertools.combinations(primes, 4)
prime_multiples = [a * b * c * d for (a, b, c, d) in prime_combinations]

for N in prime_multiples:
    count = count + limit // N

print(count)

1852825865389314


Right now, `count` feels like an arbitrary number, and it may seem like we're done. However, there is more! We need to consider all the different N that have some common multiples.

For example, if one `N` is 5, we count all of its multiples: 5, 10, **15**, 20...

If another `N` is 3, we count: 3, 6, 9, 12, **15**, 18...

This means that we're overcounting certain numbers, since they're multiples of different N. *And we're overcounting the multiples of these certain numbers too.* In this example, the number 30 is also counted in both lists. So is 45. This means that we need to find a list of the least common multiples between various `N` and we need to subtract them and their multiples from our `count`.

Computing all the LCMs for so many numbers however is going to be very computationally expensive again. A naive approach I tried ran for 30 mins straight without yielding any answer at all. So I started looking for patterns, and I found one.

It seems like the LCMs of all the combinations of prime multiples formed by choosing 4 primes out of 5 are the same. That sentence is a bit dense, so let's unpack it.

You have 5 primes as follows, and you create all the possible prime multiples you can by choosing 4 out of the 5.

In [9]:
# Try messing with this list. Note that the prime_multiple_lcms list contains the exact same numbers
# for any 5 primes in the list below. If you change one of the numbers to a composite, the list
# contains more than one distinct LCM.
prime_test = [3, 5, 7, 11, 13]
prime_test_multiples = [a * b * c * d for a, b, c, d in itertools.combinations(prime_test, 4)]
print(prime_test_multiples)

[1155, 1365, 2145, 3003, 5005]


Now you have 5 multiples. Let's see what the LCMs are between any combination of these numbers:

In [10]:
prime_multiple_lcms = [lcm(a, b) for a, b in itertools.combinations(prime_test_multiples, 2)]
print(prime_multiple_lcms)

[15015, 15015, 15015, 15015, 15015, 15015, 15015, 15015, 15015, 15015]


What this seems to imply is that our prime multiple lists look something like this:

    1155, 2310...15015...30030...
    1365, 2730...15015...30030...
    2145, 4290...15015...30030...
    ...
    ...

That means that this specific LCM (15015) and all of its multiples under `limit` are being counted **5** times, once in each list of prime multiples. We need to count it only once, so we can simply subtract this LCM from our count **4** times, and our problem will be done.

To proceed, we need to think about how we can get all of these LCMs. One way I can think of is to get all the unique 5-length combinations of primes, and then to find the LCMs for those combinations.

Here's some code that does that:

In [11]:
prime_common_combination = itertools.combinations(primes, 5)

We have all the 5-length combinations of primes that we need. We need to just calculate the LCM which they all share. Let's do that:

In [12]:
lcm_to_remove = []
# For every combination of 5 primes
for a, b, c, d, e in prime_common_combination:
    # Add the LCM of of any 2 different prime multiples
    lcm_to_remove.append(lcm(a * b * c * d, a * b * c * e))

Nearly there!

Remember that every LCM was counted 5 times. To count it once, we need to remove it 4 times. This applies to all the multiples of this LCM too. To see how many multiples of the LCM there are, we can use the same trick as before.

In [13]:
for cur_lcm in lcm_to_remove:
    count = count - (4 * (limit // cur_lcm))
print(count)

313785311322006


And I think that is it! Sadly, I can't know for sure, since I only have the question, not the answer.

##Concerns
Some people said that I failed to account for the part of the question that mentioned that we're counting numbers with **at least** 4 primes. It may seem that way, however this solution does account for all numbers with more than 4 prime factors.

For example, let's say we have this number: `N = 2*3*5*7*11*13*17`
This number has 7 prime factors. In our solution, one of the prime multiples is `X = 2*3*5*7`. We count all the multiples of `X` under `limit`. *All* multiples. `X * 11*13*17` is also a multiple of `X`, which means that we've already accounted for that.

##Contact
If you think I made a mistake somewhere, feel free to [email me](mailto:amaan.cheval@gmail.com) or [tweet at me](https://twitter.com/AmaanC).